In [1]:
import xarray as xr
from dask.distributed import Client

In [2]:
mesh = xr.open_dataset('/home/sallen/MEOPAR/grid/mesh_mask202108.nc')
tmask = 1 - mesh.tmask[0]
volume = ((1-tmask) * mesh.e3t_0[0] * mesh.e1t[0] * mesh.e2t[0]).rename({'z':'deptht'})

In [3]:
chunk_sizes = {'time_counter': 1, 'deptht': 40, 'gridY': 898, 'gridX': 398}

In [4]:
ww_out_p1 = xr.open_dataset('/data/sallen/results/MEOPAR/Scrubberwater/01jan19_nn/SalishSea_1d_20190101_20190630_scrb_T.nc', chunks=chunk_sizes)
ww_out_p2 = xr.open_dataset('/data/sallen/results/MEOPAR/Scrubberwater/01jul19_nn/SalishSea_1d_20190701_20191231_scrb_T.nc', chunks=chunk_sizes)

In [1]:
client=Client(processes=True, n_workers=4, local_directory='/tmp/cdonaldson')  # 4 is a good choice for n_workers even on salish, if I was the only person logged on or asked for space, could go as high as 16 but... be conscious of this
# threads_per_worker = 1 if needed, netcdf is not threadsafe now so be realiable by specifying 1 (at expense of just some efficiency)

NameError: name 'Client' is not defined

In [5]:
ww_out = xr.concat([ww_out_p1, ww_out_p2], dim='time_counter')

In [6]:
ww_volume = ww_out[['no_decay', 'slow_decay', 'medium_decay', 'fast_decay']] * volume

In [ ]:
ww_volume_summed = ww_volume.sum(dim=['deptht', 'x', 'y'])

: 

In [ ]:
ww_volume_summed.to_netcdf('/ocean/cdonaldson/scrubbers/calcs/SalishSea_1d_20190101_20191231_scrb_T_accumulation.nc', engine='netcdf4')

In [ ]:
client.close()